In [1]:
import numpy as np
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
import pydub
from pydub.playback import play
import IPython.display as ipd
import os
output_notebook()

Loading BokehJS ...

In [60]:
# This class defines the core Guitar Effects object. 
# It contains functions to read and write audio files.
# It also contains all the different functions implementing various guitar effects

class GEcore():
    
    def __init__(self):
        self.effectname = ''
        self.audiofilename = ''
        self.framerate = []
        self.signal = []
        self.read_audiofile()
        
    def read_audiofile(self):
        name = input('Enter the audio filename you want to read including the extension: ')
        filename, file_ext = os.path.splitext(name)
        filename = os.getcwd() + '/audiofiles/' + name
        self.audiofilename = filename
        audiofile = pydub.AudioSegment.from_file(filename, file_ext)
        audiofile = audiofile.fade_out(2000)
        self.framerate = audiofile.frame_rate
        songdata = []  # Empty list for holding audio data
        channels = []  # Empty list to hold data from separate channels
        songdata = np.frombuffer(audiofile._data, np.int16)
        for chn in range(audiofile.channels):
            channels.append(songdata[chn::audiofile.channels])  # separate signal from channels
        self.signal = np.sum(channels, axis=0) / len(channels)  # Averaging signal over all channels
        p_audiofile = self.plot_signal([self.signal])
        show(p_audiofile)
        
    def plot_signal(self, audio_signal):
        p = figure(plot_width=900, plot_height=500, title='Audio Signal', 
                   x_axis_label='Time (s)', y_axis_label='Amplitude (arb. units)')
        time = np.linspace(0, np.shape(audio_signal)[1] / self.framerate, np.shape(audio_signal)[1])
        m = int(np.shape(audio_signal)[1] / 2000)
        for n in range(np.shape(audio_signal)[0]):
            p.line(time[0::m], audio_signal[n][0::m])
        return(p)
    
    def delay(self, input_signal):
        delaytime = int(input('Enter the delay you want to add in miliseconds: '))
        gain = float(input('Enter the gain (a number betweeen 0 and 1): '))
        num = int(delaytime * 1e-3 * self.framerate)
        delaysig = np.roll(input_signal, num)
        delaysig[:num] = 0
        output_signal = input_signal + gain * delaysig
        p_delay = self.plot_signal([input_signal, output_signal])
        show(p_delay)
        return output_signal
    
    def flanger(self, input_signal):
        maxdelay = int(input('Enter the maximum delay you want to add in milliseconds: '))
        fflanger = float(input('Enter the frequency of delay oscillation in Hz: '))
        gain = float(input('Enter the gain (a number betweeen 0 and 1): '))
        num = int(maxdelay * 1e-3 * self.framerate)
        output_signal = np.zeros(len(input_signal))
        delaysig = np.zeros(num)
        for n in range(len(input_signal)):
            d = int(0.5 * num * (1 + np.sin(2 * np.pi * fflanger * n / self.framerate)))
            if d < n:
                output_signal[n] = input_signal[n] + gain * input_signal[n-d]
            else:
                output_signal[n] = input_signal[n] 
        p_flanger = self.plot_signal(output_signal)
        show(p_flanger)
        return output_signal
    
    def overdrive(self, input_signal):
        normsig = input_signal / np.max(np.absolute(input_signal))
        output_signal = np.zeros(len(input_signal))
        for n in range(len(input_signal)):
            if np.absolute(normsig[n]) < 0.3333:
                output_signal[n] = 2 * normsig[n]
            if np.absolute(normsig[n]) >= 0.3333:
                if normsig[n] > 0:
                    output_signal[n] = (3 - (2 - 3 * normsig[n])**2) / 3
                if normsig[n] < 0:
                    output_signal[n] = -(3 - (2 - 3 * np.absolute(normsig[n]))**2) / 3
            if np.absolute(normsig[n]) > 2 / 3:
                if normsig[n] > 0:
                    output_signal[n] = 1
                if normsig[n] < 0:
                    output_signal[n] = -1
        p_overdrive = self.plot_signal(output_signal)
        show(p_overdrive)
        return output_signal

In [61]:
ge = GEcore()
ipd.Audio(ge.signal, rate=ge.framerate)

Enter the audio filename you want to read including the extension: cleanchord-A.wav


In [63]:
adelay = ge.delay(ge.signal)
ipd.Audio(adelay, rate=ge.framerate)

Enter the delay you want to add in miliseconds: 2000
Enter the gain (a number betweeen 0 and 1): 1


In [6]:
aflanger = ge.flanger(adelay)
ipd.Audio(aflanger, rate=ge.framerate)

Enter the maximum delay you want to add in milliseconds: 15
Enter the frequency of delay oscillation in Hz: 1.5
Enter the gain (a number betweeen 0 and 1): 0.8


In [32]:
aoverdrive = ge.overdrive(ge.signal)
ipd.Audio(aoverdrive, rate=ge.framerate)

In [51]:
test = [ge.signal, adelay]
test[0][1]

282.5